In [118]:
import numpy as np
import os,glob
import scipy.ndimage as sni
import sys
import re
import itertools
from shutil import copytree
from astropy.wcs import WCS
from astropy.io import fits
from astropy.utils.data import get_pkg_data_filename
import astropy.units as u
import matplotlib.pyplot as plt
import numpy as np
from astropy.visualization import make_lupton_rgb
import aplpy
from astropy.nddata import Cutout2D
from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.visualization.wcsaxes import SphericalCircle
from matplotlib.patches import Arrow
from photutils import SkyCircularAnnulus
from photutils import SkyCircularAperture
from photutils import CircularAperture
from photutils import CircularAnnulus
from photutils import SkyEllipticalAperture
from photutils import SkyEllipticalAnnulus
from photutils import EllipticalAperture
from photutils import EllipticalAnnulus
from photutils import aperture_photometry
import numpy.ma as ma
import math
import shutil
import pandas as pd

In [119]:
Dir='/home/heh15/workingspace/Arp240/NGC5257/ratio/'
scriptDir=Dir+'script/measure_1213/'
workDir=Dir+'mask/'
imageDir=Dir+'image/'
logDir=Dir+'log/'
ratioDir=Dir+'1213/contsub/'
maskDir=Dir+'mask/'

In [120]:
# function
def fits_import(fitsimage):
    hdr = fits.open(fitsimage)[0].header
    wcs = WCS(hdr).celestial
    data=fits.open(fitsimage)[0].data[0][0]
    position=SkyCoord(dec=50.4167*u.arcmin,ra=204.9706*u.degree,frame='icrs')
    size=u.Quantity((54,42),u.arcsec)
    cut=Cutout2D(data=data,position=position,size=size,wcs=wcs)
    data_cut=cut.data
    wcs_cut=cut.wcs
    data_masked=np.ma.masked_invalid(data_cut)
    return wcs_cut, data_masked

def masked_convert(data_masked,region_masked):
    data_mask=data_masked.mask
    region_mask=np.ma.make_mask(region_masked==0)
    region_mask=np.ma.mask_or(data_mask,region_mask)
    data_region=np.ma.masked_where(region_mask,data_masked)
    return data_region

def flux_mask_get(data_region,rms,chans,chan_width):
    flux=np.ma.sum(data_region)/beam_area_pix
    chans_tmp=chans+np.zeros((np.shape(data_region)[0],np.shape(data_region)[1]))
    error=np.sqrt(chans_tmp)*rms*chan_width/math.sqrt(beam_area_pix)
    error_masked=np.ma.masked_where(data_region.mask,error)
    uncertainty=math.sqrt(np.ma.sum(np.power(error_masked,2)))
    return flux, uncertainty

def ratio_cal(flux1,flux2,uncertainty1,uncertainty2):
    ratio=flux1/flux2
    uncertainty=math.sqrt((uncertainty1/flux1)**2+(uncertainty2/flux2)**2+2*0.05**2)*ratio
    return ratio, uncertainty

In [121]:
beam_area=3.789*2.989*1.1331
beam_area_pix=beam_area/0.09
regions=['center','hinge','south','rest','all']
values=['12CO10 flux','12CO10 uncertainty','13CO10 flux', '13CO10 uncertainty', '12CO21 flux',
        '12CO21 uncertainty','12/13','12/13 uncertainty','21/10','21/10 uncertainty']
CO13=['13CO10 flux', '13CO10 uncertainty']
ratio_table=pd.DataFrame(index=regions,columns=values)

values2=['12CO10 intensity','12CO10 uncertainty','13CO10 intensity','13CO10 uncertainty',
         '12CO21 intensity','12CO21 uncertainty']
intensities=pd.DataFrame(columns=values2,index=regions)
peaks=pd.DataFrame(columns=values2,index=['center'])

## 12CO 1-0

In [122]:
fitsimage=imageDir+'12CO10/NGC5257_12CO10_combine_contsub_uvrange_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['center']=flux
ratio_table['12CO10 uncertainty']['center']=uncertainty
peaks['12CO10 intensity']['center']=np.nanmax(region)
peaks['12CO10 uncertainty']['center']= peaks['12CO10 intensity']['center']*uncertainty/flux


# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['hinge']=flux
ratio_table['12CO10 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['south']=flux
ratio_table['12CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0023
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO10 flux']['rest']=flux
ratio_table['12CO10 uncertainty']['rest']=uncertainty

In [123]:
# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['center']=intensity
intensities['12CO10 uncertainty']['center']=intensity*ratio_table['12CO10 uncertainty']['center']\
                                  /ratio_table['12CO10 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['hinge']=intensity
intensities['12CO10 uncertainty']['hinge']=intensity*ratio_table['12CO10 uncertainty']['hinge']\
                                  /ratio_table['12CO10 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['south']=intensity
intensities['12CO10 uncertainty']['south']=intensity*ratio_table['12CO10 uncertainty']['south']\
                                  /ratio_table['12CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO10 intensity']['rest']=intensity
intensities['12CO10 uncertainty']['rest']=intensity*ratio_table['12CO10 uncertainty']['rest']\
                                  /ratio_table['12CO10 flux']['rest']

## 13CO 1-0 

In [124]:
fitsimage=imageDir+'13CO10/NGC5257_13CO10_12m_uvrange_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

Import the count channel map for 13CO1-0

In [125]:
fitsimage=imageDir+'13CO10/NGC5257_13CO10_12m_uvrange_smooth_co32_nchan.fits'
wcs=fits_import(fitsimage)[0]
chans_masked=fits_import(fitsimage)[1]
chans=chans_masked.data

In [126]:
# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['center']=flux
ratio_table['13CO10 uncertainty']['center']=uncertainty
peaks['13CO10 intensity']['center']=np.nanmax(region)
peaks['13CO10 uncertainty']['center']= peaks['13CO10 intensity']['center']*uncertainty/flux

# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['hinge']=flux
ratio_table['13CO10 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['south']=flux
ratio_table['13CO10 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['13CO10 flux']['rest']=flux
ratio_table['13CO10 uncertainty']['rest']=uncertainty

Calculate the intensity of the 13CO 1-0

In [127]:
# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['center']=intensity
intensities['13CO10 uncertainty']['center']=intensity*ratio_table['13CO10 uncertainty']['center']\
                                  /ratio_table['13CO10 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['hinge']=intensity
intensities['13CO10 uncertainty']['hinge']=intensity*ratio_table['13CO10 uncertainty']['center']\
                                  /ratio_table['13CO10 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['south']=intensity
intensities['13CO10 uncertainty']['south']=intensity*ratio_table['13CO10 uncertainty']['south']\
                                  /ratio_table['13CO10 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.00073
chan_width=20
chans=25
intensity=np.ma.mean(region)
intensities['13CO10 intensity']['rest']=intensity
intensities['13CO10 uncertainty']['rest']=intensity*ratio_table['13CO10 uncertainty']['rest']\
                                  /ratio_table['13CO10 flux']['rest']

## 12CO21 

In [128]:
fitsimage=imageDir+'12CO21/NGC5257_12CO21_combine_contsub_uvtaper_smooth_co32_pbcor_mom0.fits'
wcs=fits_import(fitsimage)[0]
data_masked=fits_import(fitsimage)[1]

In [129]:
# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['center']=flux
ratio_table['12CO21 uncertainty']['center']=uncertainty
peaks['12CO21 intensity']['center']=np.nanmax(region)
peaks['12CO21 uncertainty']['center']= peaks['12CO21 intensity']['center']*uncertainty/flux

# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['hinge']=flux
ratio_table['12CO21 uncertainty']['hinge']=uncertainty


# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['south']=flux
ratio_table['12CO21 uncertainty']['south']=uncertainty

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
rms=0.0098
chan_width=10
chans=50
flux=flux_mask_get(region,rms,chans,chan_width)[0]
uncertainty=flux_mask_get(region,rms,chans, chan_width)[1]
ratio_table['12CO21 flux']['rest']=flux
ratio_table['12CO21 uncertainty']['rest']=uncertainty

In [130]:
# center
maskimage=maskDir+'center_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['center']=intensity
intensities['12CO21 uncertainty']['center']=intensity*ratio_table['12CO21 uncertainty']['center']\
                                  /ratio_table['12CO21 flux']['center']

# hinge
maskimage=maskDir+'hinge_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['hinge']=intensity
intensities['12CO21 uncertainty']['hinge']=intensity*ratio_table['12CO21 uncertainty']['center']\
                                  /ratio_table['12CO21 flux']['hinge']

# south
maskimage=maskDir+'south_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['south']=intensity
intensities['12CO21 uncertainty']['south']=intensity*ratio_table['12CO21 uncertainty']['south']\
                                  /ratio_table['12CO21 flux']['south']

# rest
maskimage=maskDir+'rest_SFR_co32_mask.fits'
region_masked=fits_import(maskimage)[1]
region= masked_convert(data_masked,region_masked)
intensity=np.ma.mean(region)
intensities['12CO21 intensity']['rest']=intensity
intensities['12CO21 uncertainty']['rest']=intensity*ratio_table['12CO21 uncertainty']['rest']\
                                  /ratio_table['12CO21 flux']['rest']

In [131]:
ratio_table['12/13']=ratio_table['12CO10 flux']/ratio_table['13CO10 flux']
fraction=((ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2+
                                  (ratio_table['13CO10 uncertainty']/ratio_table['13CO10 flux'])**2)**0.5
ratio_table['12/13 uncertainty']=ratio_table['12/13']*fraction

In [132]:
ratio_table['21/10']=ratio_table['12CO21 flux']/ratio_table['12CO10 flux']*115.27**2/230.54**2
fraction=((ratio_table['12CO21 uncertainty']/ratio_table['12CO21 flux'])**2+
                                  (ratio_table['12CO10 uncertainty']/ratio_table['12CO10 flux'])**2)**0.5
ratio_table['21/10 uncertainty']=ratio_table['21/10']*fraction
print(ratio_table)

       12CO10 flux 12CO10 uncertainty 13CO10 flux 13CO10 uncertainty  \
center      23.509           0.252612     1.42562           0.068186   
hinge      24.1989           0.393331    0.689181          0.0560438   
south      3.98693           0.288601   0.0757942          0.0205688   
rest       64.7295           0.768531     3.24164           0.305488   
all            NaN                NaN         NaN                NaN   

       12CO21 flux 12CO21 uncertainty    12/13 12/13 uncertainty     21/10  \
center     79.3149            1.07635  16.4904          0.808378  0.843453   
hinge      80.5337            1.67593  35.1126           2.91181  0.831997   
south      20.5782            1.25544   52.602           14.7741   1.29035   
rest       213.694            3.31295  19.9681           1.89665  0.825336   
all            NaN                NaN      NaN               NaN       NaN   

       21/10 uncertainty  
center         0.0145998  
hinge          0.0219695  
south           0

In [133]:
print(intensities)
print(peaks)

       12CO10 intensity 12CO10 uncertainty 13CO10 intensity  \
center          9.74432           0.104706          0.59091   
hinge           4.13719          0.0672462         0.143456   
south            1.2661           0.091649        0.0600398   
rest            2.89871          0.0344163         0.185107   
all                 NaN                NaN              NaN   

       13CO10 uncertainty 12CO21 intensity 12CO21 uncertainty  
center          0.0282626          32.8755           0.446139  
hinge           0.0141932          13.7685           0.184019  
south           0.0162934          6.26956           0.382495  
rest            0.0174442          9.34942           0.144946  
all                   NaN              NaN                NaN  
       12CO10 intensity 12CO10 uncertainty 13CO10 intensity  \
center           18.633           0.200218          1.31515   

       13CO10 uncertainty 12CO21 intensity 12CO21 uncertainty  
center          0.0629021          63.6702    